<a href="https://colab.research.google.com/github/Kushagratandon12/Introduction_to_tensorflow/blob/master/Natural_Launguage_Processing_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [56]:
print(tf.__version__)
!pip install -q tensorflow-datasets

2.3.0


# Learning Natural Language Processing (NLP)

In [37]:
sentences = [
             'Hi My Name Is Kushagra',
             'Hi My Name Is Samarth',
             'I Love My Dog',
             'You Love Dog!'
]

In [38]:
#creating an instance for the tokenizer 
tokenizer = Tokenizer(num_words=100,oov_token="<OOV>")
print(tokenizer)

In [39]:
tokenizer.fit_on_texts(sentences)
words_index = tokenizer.word_index
print(words_index)

{'<OOV>': 1, 'my': 2, 'hi': 3, 'name': 4, 'is': 5, 'love': 6, 'dog': 7, 'kushagra': 8, 'samarth': 9, 'i': 10, 'you': 11}


In [40]:
sequence = tokenizer.texts_to_sequences(sentences)
print(sequence)

[[3, 2, 4, 5, 8], [3, 2, 4, 5, 9], [10, 6, 2, 7], [11, 6, 7]]


In [50]:
padding = pad_sequences(sequence,maxlen=5)
print(padding)

[[ 3  2  4  5  8]
 [ 3  2  4  5  9]
 [ 0 10  6  2  7]
 [ 0  0 11  6  7]]


# IMDB MOVIE REVIEW PROJECT


In [57]:
import tensorflow_datasets as tfds

In [58]:
imdb , info = tfds.load("imdb_reviews" , with_info=True,as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1YUFV/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1YUFV/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteB1YUFV/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [66]:
test_data , train_data = imdb['test'] , imdb['train']
print(type(test_data))
print(type(train_data))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>
<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [85]:
training_sentences = []
training_labels = []
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

print(type(training_sentences))

<class 'list'>


In [89]:
testing_sentences=[]
testing_labels=[]
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [90]:
training_labels_final =np.array(training_labels)
testing_labels_final = np.array(testing_labels) 

In [106]:
# tokenizer = Tokenizer(num_words=10000,oov_token="<OOV>")
# tokenizer.fit_on_texts(training_sentences)
# words_index = tokenizer.word_index
# sequence = tokenizer.texts_to_sequences(training_sentences)
# padded = pad_sequences(sequence , padding='post',maxlen=120)
# print(words_index)

In [107]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [110]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [109]:
model = tf.keras.Sequential([    
                             tf.keras.layers.Embedding(10000,16,input_length=120),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(6,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
                             
])
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [104]:
model.fit(padded,
          training_labels_final,
          epochs=16,
          validation_data=(testing_padded,testing_labels_final))

Epoch 1/16
782/782 [==============================] - 4s 5ms/step - loss: 0.5327 - accuracy: 0.7632 - val_loss: 0.3708 - val_accuracy: 0.8455
Epoch 2/16
782/782 [==============================] - 4s 5ms/step - loss: 0.3011 - accuracy: 0.8790 - val_loss: 0.3201 - val_accuracy: 0.8628
Epoch 3/16
782/782 [==============================] - 4s 5ms/step - loss: 0.2443 - accuracy: 0.9021 - val_loss: 0.3188 - val_accuracy: 0.8624
Epoch 4/16
782/782 [==============================] - 4s 5ms/step - loss: 0.2115 - accuracy: 0.9198 - val_loss: 0.3327 - val_accuracy: 0.8570
Epoch 5/16
782/782 [==============================] - 3s 4ms/step - loss: 0.1873 - accuracy: 0.9310 - val_loss: 0.3539 - val_accuracy: 0.8521
Epoch 6/16
782/782 [==============================] - 3s 4ms/step - loss: 0.1676 - accuracy: 0.9398 - val_loss: 0.3734 - val_accuracy: 0.8490
Epoch 7/16
782/782 [==============================] - 4s 5ms/step - loss: 0.1519 - accuracy: 0.9465 - val_loss: 0.4043 - val_accuracy: 0.8457
Epoch 

In [105]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [113]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

[-0.09067859 -0.06638046 -0.1156228   0.06845208 -0.19336942 -0.00304176
  0.20963508  0.06484628 -0.03394631 -0.09478562  0.08967552  0.07388046
 -0.02438696  0.10826803 -0.00640544 -0.10164997]


In [112]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>